<a href="https://colab.research.google.com/github/fjme95/calculo-optimizacion/blob/main/Semana%205/Back_Propagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/programmingLearner/my-Back-Propagation/blob/19094029f9af63d789b4ce74c9b5dcce9d6aa71b/mybp.py#L274

In [3]:
import numpy as np
from queue import Queue

In [1]:
# error handling function
def error(warning):
    print(warning)
    # exit()

# progress reporting function, may be disabled after debugging
def progress(report):
    print(report)
    return


class node:
    # constructor
    def __init__(self, operator, left_child, right_child = None, is_output = False):
        # declaration of variables of this class
        self.forward = 0    # forward value
        self.backward = 0   # output gradient w.r.t. this node
        self.parents = []   # pointers to parents nodes
        self.left_child = None  # pointers to left child nodes
        self.right_child = None # pointers to right child nodes
        self.operator = ''  # operator type, e.g., input, const, +, *, sin, ...
                            # operator is either input, const, unary or binary
        self.cnt_parents = 0    # how many parents it has
        self.cnt_ref = 0    # reference count, how many gradients w.r.t. this
                            # node is added to self.backward
        self.init = False   # if the node is fed with a value
        self.bp_ed = False  # if back-prop from this node to its children nodes
                            # has completed
        self.is_output = is_output  # if the node is the output node
        self.visited = 0    # for use in clear function when traversing

        # initiate
        self.check_valid_op(operator)
        self.operator = operator
        self.left_child = left_child
        self.right_child = right_child
        if operator in {'input', 'const'}:
            return
        if self.left_child == None:
            error('Error: invalid operator without child!')
        self.left_child.parents.append(self)
        self.left_child.cnt_parents += 1
        if operator in {'+', '*'}:  # binary operator
            if self.right_child == None:
                error('Error: invalid binary operator without child!')
            self.right_child.parents.append(self)
            self.right_child.cnt_parents += 1

    # set as output node
    def set_as_output(self):
        self.is_output = True

    # clear flags previously in the nodes
    def clear(self):
        self.forward = 0
        self.backward = 0
        self.cnt_ref = 0
        self.init = False
        self.bp_ed = False

    def check_valid_op(self, operator):
        if operator not in {'input', 'const', '+', '*', 'sin', 'cos', 'tan',
                            'exp', 'log', 'neg', 'inv', 'relu'}:
            error('Error: invalid operator type: ' + operator + ' !')

    def check_initiated(self):
        if not self.init:
            error('Error: attempting to run graph without initializing!')

    # feed value, initiate
    def feed(self, value):
        if self.operator not in {'input', 'const'}:
            error('Error: attempting to feed value to non-input-or-const node!')
        self.forward = value

    # compute forwardly: compute the value of this node forwarding
    # this function will return the node's parents nodes who is ready to compute forwardly
    def compute_forward(self):
        # valid check
        if self.operator not in {'input', 'const'}:
            if self.init == True:
                error('Error: attempting to recalculate forwardly.')
        # compute
        if self.operator in {'+', '*'}: # binary operator
            self.left_child.check_initiated()
            self.right_child.check_initiated()
            if self.operator == '+':
                self.forward = self.left_child.forward + self.right_child.forward
            else:
                self.forward = self.left_child.forward * self.right_child.forward
        elif self.operator not in {'input', 'const'}:   # unary operator
            self.left_child.check_initiated()
            if self.operator == 'sin':
                self.forward = np.sin(self.left_child.forward)
            elif self.operator == 'cos':
                self.forward = np.cos(self.left_child.forward)
            elif self.operator == 'tan':
                self.forward = np.tan(self.left_child.forward)
            elif self.operator == 'exp':
                self.forward = np.exp(self.left_child.forward)
            elif self.operator == 'log':
                if self.left_child.forward <= 0:
                    error('Error: attempting to feed a non-positive number into log!')
                self.forward = np.log(self.left_child.forward)
            elif self.operator == 'neg':
                self.forward = 0 - self.left_child.forward
            elif self.operator == 'inv':
                if self.left_child.forward == 0:
                    error('Error: attempting to feed 0 into inverse!')
                self.forward = 1.0 / self.left_child.forward
            elif self.operator == 'relu':
                if self.left_child.forward > 0:
                    self.forward = self.left_child.forward
                else:
                    self.forward = 0
            else:
                error('Error: invalid operator type while forwarding!')
        self.init = True    # this should be in advance of find parents,
                            # in case, e.g. b = a + a
        # find parents ready for forwarding
        list = []
        tmp = None
        for node in self.parents:
            if tmp == node: # in case like b = a + a
                continue
            tmp = node
            if node.operator in {'+', '*'}:
                if node.left_child == self:
                    if node.right_child.init == True:
                        list.append(node)
                else:
                    if node.left_child.init == True:
                        list.append(node)
            else:
                list.append(node)
        return list

    # computing gradient w.r.t the child of this node, adding it to that
    # this function will return the node's children nodes who is ready to back-prop
    def back_prop(self):
        # special cases
        if self.is_output == True:
            if self.init == False:
                error('Error: attempting to back-prop without forwarding in advance!')
            self.backward = 1
        if self.bp_ed == True:
            error('Error: attempting to back-prop twice from the node!')
        if self.cnt_parents != self.cnt_ref:
            error('Error: attempting to back-prop in a wrong order!')
        if self.operator in {'input', 'const'}:
            self.bp_ed = True
            return []
        # compute
        if self.operator in {'+', '*'}:
            if self.operator == '+':
                self.left_child.backward += self.backward
                self.right_child.backward += self.backward
            else:
                self.left_child.backward += self.backward * self.right_child.forward
                self.right_child.backward += self.backward * self.left_child.forward
            self.left_child.cnt_ref += 1
            self.right_child.cnt_ref += 1
        else:
            if self.operator == 'sin':
                self.left_child.backward += self.backward * np.cos(self.left_child.forward)
            elif self.operator == 'cos':
                self.left_child.backward -= self.backward * np.sin(self.left_child.forward)
            elif self.operator == 'tan':
                self.left_child.backward += self.backward / ((np.cos(self.left_child.forward)) ** 2)
            elif self.operator == 'exp':
                self.left_child.backward += self.backward * np.exp(self.left_child.forward)
            elif self.operator == 'log':
                self.left_child.backward += self.backward / self.left_child.forward
            elif self.operator == 'neg':
                self.left_child.backward -= self.backward
            elif self.operator == 'inv':
                self.left_child.backward -= self.backward / (self.left_child.forward ** 2)
            elif self.operator == 'relu':
                if self.left_child.forward > 0:
                    self.left_child.backward += self.backward
            else:
                error('Error: invalid operator type while back-prop!')
            self.left_child.cnt_ref += 1
        self.bp_ed = True
        # find children who are ready to back-prop: (considering b = a + a)
        list = []
        if self.left_child.cnt_ref == self.left_child.cnt_parents:
            list.append(self.left_child)
        if self.operator in {'+', '*'}:
            if self.right_child.cnt_ref == self.right_child.cnt_parents:
                if self.right_child != self.left_child:
                    list.append(self.right_child)
        return list

    def __repr__(self):
        return f"forward: {self.forward}\nbackward: {self.backward}"

# computational graph presented as DAG
class graph:

    # constructor
    def __init__(self, input_nodes, const_nodes, output_nodes):
        progress('Welcome to use zty automatic back-prop program, ' +
              'make sure that you have indicated which unique node is the output.')
        self.input_nodes = input_nodes
        self.const_nodes = const_nodes
        self.output_nodes = output_nodes
        self.visited_ind = 1    # the nodes in the graph is visited during a traversing
                                # if its visited bit is the same as self.visited indicator
        progress('Progress: computational graph imported.')

    # feed value, initiate
    def feed(self, input_value_list, const_value_list):
        l = len(input_value_list)
        if l != len(self.input_nodes):
            error('Error: input value list does not match!')
        for i in range(l):
            self.input_nodes[i].feed(input_value_list[i])
        l = len(const_value_list)
        if l != len(self.const_nodes):
            error('Error: const value list does not match!')
        for i in range(l):
            self.const_nodes[i].feed(const_value_list[i])
        progress('Progress: computational graph fed.')

    # clear flags previously in the DAG
    def clear(self):
        queue = Queue()
        for elm in self.output_nodes:
            queue.put(elm)
            elm.visited = self.visited_ind
        while not queue.empty():
            # print(queue.queue)
            node = queue.get()
            node.clear()
            if node.left_child is not None:
                if node.left_child.visited != self.visited_ind:
                    queue.put(node.left_child)
                    node.left_child.visited = self.visited_ind
            if node.right_child is not None:
                if node.right_child.visited != self.visited_ind:
                    queue.put(node.right_child)
                    node.right_child.visited = self.visited_ind
        self.visited_ind = 1 - self.visited_ind
        progress('Progress: computational graph cleared.')

    # compute forward
    def compute_forward(self):
        queue = Queue()
        for elm in self.input_nodes:
            queue.put(elm)
        for elm in self.const_nodes:
            queue.put(elm)
        while not queue.empty():
            node = queue.get()
            preparing = node.compute_forward()
            for elm in preparing:
                queue.put(elm)
        progress('Progress: computational graph forwarded.')

    # back prop
    def back_prop(self):
        queue = Queue()
        for elm in self.output_nodes:
            queue.put(elm)
        while not queue.empty():
            # print(queue.queue)
            node = queue.get()
            preparing = node.back_prop()
            for elm in preparing:
                queue.put(elm)
        progress('Progress: computational graph back-propagated.')



In [4]:
x1 = node('input', None)
x2 = node('input', None)

c1 = node('const', None)

A = node('+', x1, x2)
B = node('*', A, c1, is_output=True)

G = graph([x1,x2], [c1], [B])

x_init = [1, 2]
const_init = [4]

G.feed(x_init, const_init)

G.compute_forward()
G.back_prop()

Welcome to use zty automatic back-prop program, make sure that you have indicated which unique node is the output.
Progress: computational graph imported.
Progress: computational graph fed.
Progress: computational graph forwarded.
Progress: computational graph back-propagated.


In [8]:
x1.backward

4

In [23]:
x1 = node('input', None)
x2 = node('input', None)

w0 = node('input', None)
w1 = node('input', None)
w2 = node('input', None)

one = node('const', None)
sigmoid = node('inv', node('+',
    node(
        'exp',
         node(
             'neg', 
               )
              )
         ), 
     one
    ), is_output=True)
G = graph([x1,x2], [w0, w1, w2, one], [sigmoid])
x_init = [.5, .2]
const_init = [1.0, 2.0, 3.0, 1]
G.feed(x_init, const_init)
G.compute_forward()
G.back_prop()

Welcome to use zty automatic back-prop program, make sure that you have indicated which unique node is the output.
Progress: computational graph imported.
Progress: computational graph fed.
Progress: computational graph forwarded.
Progress: computational graph back-propagated.


In [5]:
sigmoid.forward

0.9308615796566533

In [6]:
real_sig = 1/(1+np.exp(-(1 + 2*.5 + 3*.2)))
real_sig

0.9308615796566533

In [7]:
real_sig * (1-real_sig) * 1

0.06435829917577342

In [8]:
w0.backward

0.06435829917577351

In [24]:
G.clear()

Progress: computational graph cleared.


In [115]:
# for y = \sum_i (x_i), return y
def scalar_sum(x):
    if type(x) is not list:
        raise ValueError(f'x debe ser una lista, pero se recibió {type(x)}')
    # TODO Revisar que todos sean nodos
    n = len(x)
    if n == 1:
        return x[0]
    sum = node('+', x[0], x[1])
    for i in range(2, n):
        sum = node('+', sum, x[i])
    return sum

def linear_combination(coef, x):
    return scalar_sum([node('*', coef[i], x[i]) for i in range(len(coef))])

def sigmoid(coef, x):
    one = node('const', None)
    return one, node('inv', node('+',node('exp',node('neg', linear_combination(coef, x))), one))

def sigmod_scalar(coef, x):
    return 1/(1+np.exp(-np.sum(coef*x)))

In [116]:
x1 = node('input', None)
x2 = node('input', None)

w1 = node('input', None)
w2 = node('input', None)

one, aver = sigmoid([w1, w2], [x1, x2])
aver.set_as_output()

G = graph([x1, x2], [w1, w2, one], [aver])
x_init = [.2, .7]
const_init = [1.6, 2.0, 1]
G.feed(x_init, const_init)
G.compute_forward()
G.back_prop()

Welcome to use zty automatic back-prop program, make sure that you have indicated which unique node is the output.
Progress: computational graph imported.
Progress: computational graph fed.
Progress: computational graph forwarded.
Progress: computational graph back-propagated.


In [117]:
aver.forward

0.8481288363433407

In [118]:
sigmod_scalar(np.array(x_init), np.array(const_init[:2]))

0.8481288363433407

# Ejercicios

- Agregar tanh a las posibles operaciones de los nodos
- Codificar una regresión logística con los nodos